<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [8]:
!apt install 7zip
!python -m pip install --upgrade pip setuptools wheel
!pip install libtorrent lbry-libtorrent dropbox tomlkit
# !python -m pip install lbry-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
7zip is already the newest version (21.07+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Imports
rich.progress for concurrent progress tracking<br>
libtorrent as torrent client<br>
dropbox as storage solution<br>

In [9]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed
import logging
import time
from timeit import default_timer as timer

from pathlib import Path
from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

import libtorrent as lt
from dropbox import Dropbox, DropboxOAuth2FlowNoRedirect
import dropbox

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn

# Setup Filesystem

In [10]:
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())
dbxcfg_path = Path("/content/dbx.conf")
dbxcfg_file = TOMLFile(dbxcfg_path.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("torrents stored in a folder or are a single file"))
  magnet_document.add("show1fldr", example_folder_tor)
  magnet_document.add("movie1", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("show2fldr", example_multifiles)

  magnet_file.write(magnet_document)

if not dbxcfg_path.exists():
  dbxcfg_document = document()
  auth_table = table()
  auth_table["APP_KEY"] = ""
  auth_table["APP_SECRET"] = ""
  auth_table["REFRESH_TOKEN"] = ""
  dbxcfg_document.add("AUTHENTICATION", auth_table)

  dbxcfg_file.write(dbxcfg_document)

# Download Torrents

In [11]:
# TODO: Handle torrent files
def download_torrent(progress: Progress, media_name: str, link: str):
  MAX_FILENAME_LEN = 35
  METADATA_TIMEOUT_MS = .8
  save_path=Path("/content/Torrents/") / media_name
  if not save_path.exists():
    save_path.mkdir(parents=True)

  task = progress.add_task(
        "download",
        total=100,
        status="[yellow]Obtaining Metadata",
        filename=media_name,
        speed="0.0 kB/s",
  )
  ses = lt.session()

  # setup torrent handle
  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    progress.update(task, status="[red]Failed to parse magnet!")
    return None
  atp.save_path = str(save_path.as_posix())

  handle = ses.add_torrent(atp)
  status = handle.status()

  while not handle.status().has_metadata:
    time.sleep(METADATA_TIMEOUT_MS/1000)
    status = handle.status()

  filename =  status.name if len(status.name) < MAX_FILENAME_LEN \
              else f"{status.name[:MAX_FILENAME_LEN]}..."

  progress.update(task,
                  status="[yellow]Downloading",
                  filename=filename
  )

  # download torrent
  while not status.is_seeding:
    status = handle.status()

    progress.update(task,
                completed=status.progress * 100,
                speed=f"{status.download_rate/1000:.1f} kB/s",
    )
  progress.update(task,
                  status="[green]Complete",
                  completed=100,
  )

  return save_path

In [7]:
def zipfiles(progress, m, magnet_links):
  with ThreadPoolExecutor() as executor:
    futures = [executor.submit(download_torrent, progress, m, link) for link in magnet_links]
    save_path = Path("/content/Torrents/") / m

    for future in as_completed(futures):
      try:
        future.result()
      except Exception as e:
        print("Upload failed:", e)
        return

  subprocess.run(["7za", "a", "-v1g", f"/content/transfer/{save_path.name}.zip", save_path.as_posix()])

with Progress(
  TextColumn("{task.fields[status]}"),
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
) as progress:

  with ThreadPoolExecutor() as executor:
    media = magnet_file.read()
    futures = []
    for m in media:
      magnet_links = media[m]["magnets"]

      futures.append(executor.submit(zipfiles, progress, m, magnet_links))

    for future in as_completed(futures):
      try:
          future.result()
      except Exception as e:
          print("Upload failed:", e)

Complete [Erai-raws] City the Animation - 07... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 32514.7 kB/s
Complete [Erai-raws] City the Animation - 08... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 9163.6 kB/s

In [ ]:
def oauth2(APP_KEY, APP_SECRET):
  auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY,
                                          consumer_secret=APP_SECRET,
                                          scope=['files.metadata.write'],
                                          token_access_type="offline"
  )

  authorize_url = auth_flow.start()
  print("1. Go to: " + authorize_url)
  print("2. Click \"Allow\" (you might have to log in first).")
  print("3. Copy the authorization code.")
  auth_code = input("Enter the authorization code here: ").strip()

  try:
      oauth_result = auth_flow.finish(auth_code)
  except Exception as e:
      print('Error: %s' % (e,))
      exit(1)

  return oauth_result

In [ ]:
dbxcfg = dbxcfg_file.read()
APP_KEY = dbxcfg["AUTHENTICATION"]["APP_KEY"]
APP_SECRET = dbxcfg["AUTHENTICATION"]["APP_SECRET"]

oauth = oauth2(APP_KEY, APP_SECRET)

#TODO: implement a way to check if the refresh token is still valid.
dbxcfg["AUTHENTICATION"]["REFRESH_TOKEN"] = oauth.refresh_token
dbxcfg_file.write(dbxcfg)

with Dropbox(app_key=APP_KEY,
                 app_secret=APP_SECRET,
                 oauth2_refresh_token=oauth.refresh_token
) as dbx:
  dbx.files_upload()